In [1]:
import glob
import os
import itertools

import pandas as pd
import numpy as np
from doit.tools import register_doit_as_IPython_magic
register_doit_as_IPython_magic()

from oudelaar_tiled_capture_2019 import config
datasets = config.datasets()
doit_db = os.path.join(config.base_folder, '.doit.db')
doit_opts = '--db-file {} --backend sqlite3'.format(doit_db)

In [2]:
atac_bams = glob.glob(config.in_data_raw('atac/*/filtered.mm9.bam'))

In [3]:
def bam_sample_name(bam_path):
    return os.path.basename(os.path.dirname(bam_path))

def bam_build(bam_path):
    return os.path.basename(bam_path).split('.')[1]

def swap_ext(input_filename, new_ext):
    """Remove the last extension from input_filename and replace it with new_ext"""
    
    root, ext = os.path.splitext(input_filename)
    return root + new_ext

In [4]:
sequenced_samples = pd.DataFrame({'path': atac_bams,
              'sample_id': [bam_sample_name(bam) for bam in atac_bams]})

In [5]:
datasets = pd.merge(datasets, sequenced_samples, on='sample_id')

datasets.head()

,sample_id,type,project,strain,population,expt_no,technical_rep,sequenced,notes,path
0,RBAT31,atac,fetal_liver_s0_s1_transition,bl6,S0_CD71_low,13,1,yes,NaN,/t1-data/user/rbeagrie/projects/oudelaar-tiled...
1,RBAT32,atac,fetal_liver_s0_s1_transition,bl6,S0_CD71_low,13,2,yes,NaN,/t1-data/user/rbeagrie/projects/oudelaar-tiled...
2,RBAT33,atac,fetal_liver_s0_s1_transition,bl6,S0_CD71_medium,13,1,yes,NaN,/t1-data/user/rbeagrie/projects/oudelaar-tiled...
3,RBAT34,atac,fetal_liver_s0_s1_transition,bl6,S0_CD71_medium,13,2,yes,NaN,/t1-data/user/rbeagrie/projects/oudelaar-tiled...
4,RBAT35,atac,fetal_liver_s0_s1_transition,bl6,S1,13,1,yes,NaN,/t1-data/user/rbeagrie/projects/oudelaar-tiled...


In [6]:
tech_rep_counts = datasets.groupby(['population', 'expt_no']).technical_rep.count()

In [7]:
tech_rep_counts

population      expt_no
HSPC            93         1
LMPP            93         1
Lin             35         2
MPP2            93         1
MPP3            93         1
S0_CD71_low     10         2
                13         2
                35         1
S0_CD71_medium  10         2
                13         2
                35         2
S1              10         2
                13         2
                35         2
S2              13         2
                35         2
S3              13         2
                35         2
Name: technical_rep, dtype: int64

In [8]:
def merged_tech_id(samples_to_merge):
    
    to_merge_paths = samples_to_merge.path.tolist()
    
    sample_ids = '-'.join([bam_sample_name(bam) for bam in to_merge_paths])
    
    population, = samples_to_merge.population.unique().tolist()
    expt_no, = samples_to_merge.expt_no.unique().tolist()
    
    merged_id = "{population}_expt{expt}_{sample_ids}".format(population=population,
                                                        expt=expt_no,
                                                        sample_ids=sample_ids)
    
    return merged_id

In [9]:
# Handle samples with two technical replicates

merged_tech_base_dir = config.in_data_intermediate('atac/merged_technical_reps')


def task_merge_bams():

    for (population, expt_no), count in tech_rep_counts[
        tech_rep_counts == 2].iteritems():

        samples = datasets[np.logical_and(
            datasets.population == population,
            datasets.expt_no == expt_no)]

        assert len(samples) == 2

        merged_id = merged_tech_id(samples)
        
        for build in 'mm9', 'mm10':

            merged_path = os.path.join(
                merged_tech_base_dir, merged_id, 'filtered.{}.bam'.format(build))

            yield {
                'name': '{}-{}'.format(merged_id, build),
                'file_dep': [p.replace('mm9', build) for p in samples.path.tolist()],
                'targets': [merged_path],
                'actions': ['mkdir -pv {}'.format(os.path.dirname(merged_path)),
                            'samtools merge -f %(targets)s %(dependencies)s']
            }
    
    

In [10]:
def linked_tech_id(sample):
    
    sample_id = bam_sample_name(sample.path)
    
    merged_id = "{population}_expt{expt}_{sample_id}".format(population=sample.population,
                                                        expt=sample.expt_no,
                                                        sample_id=sample_id)
    
    return merged_id

In [11]:
def task_link_files():

    for (population, expt_no), count in tech_rep_counts[
        tech_rep_counts == 1].iteritems():

        sample = datasets[np.logical_and(
            datasets.population == population,
            datasets.expt_no == expt_no)].iloc[0]

        merged_id = linked_tech_id(sample)

        for merged_file in 'filtered.{}.bam', 'filtered.{}.sorted.bam', 'filtered.{}.sorted.bam.bai':

            for build in 'mm9', 'mm10':

                merged_path = os.path.join(merged_tech_base_dir, merged_id, merged_file.format(build))

                yield {
                    'name': '{}-{}'.format(merged_id, merged_file.format(build)),
                    'file_dep': [os.path.join(
                                    os.path.dirname(sample.path),
                                    merged_file.format(build))],
                    'targets': [merged_path],
                    'actions': ['mkdir -pv {}'.format(os.path.dirname(merged_path)),
                                'ln -s %(dependencies)s %(targets)s']
                }

In [12]:
def task_sort_bams():
    for task in task_merge_bams():
        
        bam = task['targets'][0]
        
        yield {
                'name' : '{}-{}'.format(bam_sample_name(bam), bam_build(bam)),
                'file_dep' : [bam],
                'targets' : [swap_ext(bam, '.sorted.bam')],
                'actions' : ['samtools sort %(dependencies)s -o %(targets)s'],
            }

In [13]:
def task_index_bams():
    for task in task_sort_bams():
        
        bam = task['targets'][0]
        
        yield {
                'name' : '{}-{}'.format(bam_sample_name(bam), bam_build(bam)),
                'file_dep' : [bam],
                'targets' : [bam + '.bai'],
                'actions' : ['samtools index %(dependencies)s'],
            }

In [14]:
peak_calls_base_dir = config.in_data_intermediate('atac/merged_technical_reps/peak_calls')

def peaks_file_path(bam_path):
    sample = bam_sample_name(bam_path)
    build = bam_build(bam_path)
    return os.path.join(
        '{}_{}'.format(peak_calls_base_dir, build),
        sample, '{}_peaks.narrowPeak'.format(sample))

def task_call_peaks():
    
    for task in list(task_sort_bams()) + list(task_link_files()):
        
        bam = task['targets'][0]
    
        if not bam[-11:] == '.sorted.bam': continue
            
        yield {
                'name': '{}-{}'.format(bam_sample_name(bam), bam_build(bam)),
                'file_dep' : [bam],
                'targets' : [peaks_file_path(bam)],
                'actions' : ['mkdir -pv $(dirname %(targets)s)',
                    'macs2 callpeak -t %(dependencies)s -n {name} -f BAMPE -g 1.87e9 -q 0.1 '
                    '--outdir $(dirname %(targets)s)'.format(name=bam_sample_name(bam))],
            }

In [15]:
all_peaks_bed = config.in_data_intermediate('atac/fetal_liver_atac_peaks.{}.bed')

def task_merge_peaks():
    """doit task to merge multiple peak files into one"""
    
    for build in 'mm9', 'mm10':
    
        yield {
            'name': build,
            'file_dep' : [t['targets'][0] for t in task_call_peaks() if build in t['targets'][0]],
            'targets' : [all_peaks_bed.format(build)],
            'actions' : [('cat %(dependencies)s | cut -f 1-4 | bedtools sort | '
                          'bedtools merge -c 4 -o distinct > %(targets)s')]
        }

In [16]:
reproducible_peaks_bed = config.in_data_intermediate('atac/reproducible_fetal_liver_atac_peaks.{}.bed')

def get_reproducible_peaks(dependencies, targets):
    """Save copy of a peaks file containing only peaks found in multiple samples"""
    all_merged_peaks = pd.read_csv(
        list(dependencies)[0], sep='\t', 
        names=['chrom', 'start', 'stop', 'samples'])
    all_merged_peaks.sample_count = all_merged_peaks.samples.str.count(',') + 1
    
    reproducible_peaks = all_merged_peaks[all_merged_peaks.sample_count > 1]

    reproducible_peaks.to_csv(list(targets)[0], sep='\t', index=False, header=False)
    
def task_reproducible_peaks():
    
    for build in 'mm9', 'mm10':
    
        yield {
            'name': build,
            'file_dep': [all_peaks_bed.format(build)],
            'targets': [reproducible_peaks_bed.format(build)],
            'actions': [get_reproducible_peaks]}

In [17]:
final_peaks_table = config.in_data_processed('atac/fetal_liver_atac_counts.{}.table')

def task_counts_table():
    
    for build in 'mm9', 'mm10':
        
        bams_to_count = sorted([t['file_dep'][0] for t in task_call_peaks() if build in t['file_dep'][0]])
        
        header_line = ' '.join(
            ['chrom', 'start', 'stop', 'name'] + 
            [bam_sample_name(bam) for bam in bams_to_count])

        yield {
            'name': build,
            'file_dep': [reproducible_peaks_bed.format(build)],
            'targets': [final_peaks_table.format(build)],
            'actions': [
                'echo {} | tr -s " " "\t" > %(targets)s'.format(header_line),
                'bedtools multicov -bed %(dependencies)s -bams {} >> %(targets)s'.format(
                    ' '.join(bams_to_count))
            ]
        }

In [18]:
%doit -n 5 {doit_opts}

-- merge_bams:Lin_expt35_RBAT61-RBAT62-mm9
-- merge_bams:Lin_expt35_RBAT61-RBAT62-mm10
-- merge_bams:S0_CD71_low_expt10_RBAT42-RBAT43-mm9
-- merge_bams:S0_CD71_low_expt10_RBAT42-RBAT43-mm10
-- merge_bams:S0_CD71_low_expt13_RBAT31-RBAT32-mm9
-- merge_bams:S0_CD71_low_expt13_RBAT31-RBAT32-mm10
-- merge_bams:S0_CD71_medium_expt10_RBAT44-RBAT45-mm9
-- merge_bams:S0_CD71_medium_expt10_RBAT44-RBAT45-mm10
-- merge_bams:S0_CD71_medium_expt13_RBAT33-RBAT34-mm9
-- merge_bams:S0_CD71_medium_expt13_RBAT33-RBAT34-mm10
-- merge_bams:S0_CD71_medium_expt35_RBAT53-RBAT54-mm9
-- merge_bams:S0_CD71_medium_expt35_RBAT53-RBAT54-mm10
-- merge_bams:S1_expt10_RBAT46-RBAT47-mm9
-- merge_bams:S1_expt10_RBAT46-RBAT47-mm10
-- merge_bams:S1_expt13_RBAT35-RBAT36-mm9
-- merge_bams:S1_expt13_RBAT35-RBAT36-mm10
-- merge_bams:S1_expt35_RBAT55-RBAT56-mm9
-- merge_bams:S1_expt35_RBAT55-RBAT56-mm10
-- merge_bams:S2_expt13_RBAT37-RBAT38-mm9
-- merge_bams:S2_expt13_RBAT37-RBAT38-mm10
-- merge_bams:S2_expt35_RBAT57-RBAT58-mm